# Introduction

We will be using Colab as IDE for our project. Through colab we will be able to diectly access Google drive where our data is stored and have the advantage of GPUs which will help us with computational power.

First we will be mounting our drive to this notebook

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Install packages
Colab does not provide us with all the modules/packlages. Whenever we are using modules other than the built-in, we should install the seperately as shown below.

In [2]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=d58d7fd834bac5b8d7a7f0cf9c83ff103e96e3abb28b252ccf2318840e027a4d
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [3]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 5.4 MB/s eta 0:00:00


#Import Modules
We should import all the packages that we use in this project

In [4]:
import nltk
nltk.download('punkt')
nltk.download('all')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from nltk import tokenize
import string
from unidecode import unidecode
from collections import Counter
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding, LayerNormalization, MultiHeadAttention, Add, Bidirectional, Concatenate, Bidirectional, Attention
from keras.models import Model
from langdetect import detect

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data

#Loading Data
Once the drive was connected, we have to load the data and rename the columns if required. Then deal with negatives if there any. Since this is a english to french data set, it is less likely to have missing values but we should focus on data cleasing for sure.

In [112]:
# load the data
data = pd.read_csv("/content/drive/MyDrive/eng_-french.csv")
data = data.rename(columns={'en': 'english', 'fr': 'french'})

data = data.dropna()
data.columns = ['en', 'fr']

Let's check the schema of the data.

In [113]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175621 entries, 0 to 175620
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   en      175621 non-null  object
 1   fr      175621 non-null  object
dtypes: object(2)
memory usage: 2.7+ MB


Lets count the number of sentences in each text entry of the 'en' column of the DataFrame data, and then print a statistical summary of these counts. This can be useful for understanding the distribution of sentence lengths in the dataset, which is often an important aspect in text analysis and natural language processing.

In [114]:
# Assuming 'data' is your DataFrame and 'column_name' is the name of the column you're interested in
sentence_counts = data['en'].map(lambda x: len(tokenize.sent_tokenize(x)))
print(sentence_counts.describe())

count    175621.000000
mean          1.009281
std           0.098643
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           4.000000
Name: en, dtype: float64


In [115]:
sentence_counts = data['fr'].map(lambda x: len(tokenize.sent_tokenize(x)))
print(sentence_counts.describe())

count    175621.000000
mean          1.010039
std           0.113824
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           5.000000
Name: fr, dtype: float64


In [116]:
data_fr = '\n'.join(data['fr'].tolist())
chars_fr = sorted(list(set(data_fr)))
chars_fr

['\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '«',
 '»',
 'À',
 'Â',
 'Ç',
 'É',
 'Ê',
 'Ô',
 'à',
 'á',
 'â',
 'ç',
 'è',
 'é',
 'ê',
 'ë',
 'î',
 'ï',
 'ô',
 'ö',
 'ù',
 'û',
 'œ',
 'С',
 '\u2009',
 '\u200b',
 '–',
 '‘',
 '’',
 '…',
 '\u202f',
 '‽',
 '₂']

In [117]:
data_en = '\n'.join(data['en'].tolist())
chars_en = sorted(list(set(data_en)))
chars_en

['\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\xa0',
 '\xad',
 'º',
 'ç',
 'é',
 'ö',
 'ú',
 'а',
 '–',
 '—',
 '‘',
 '’',
 '₂',
 '€']

In [118]:

# Sample DataFrame creation for demonstration (you'll use your own DataFrame)
# data = pd.DataFrame({
#     'english': ['This is a sample text.', 'Another example text.'],
#     'french': ['Ceci est un texte d\'exemple.', 'Un autre texte d\'exemple.']
# })

# Constants
start_token = '<START>'
end_token = '<END>'
unk_token = '<UNK>'
pad_token = '<PAD>'

def clean_text(text, start_token=start_token, end_token=end_token):
    text = text.lower()  # convert uppercase to lowercase
    text = unidecode(text, errors='ignore')  # convert accented letters into unaccented letters. Ignore unknown characters.
    text = ''.join((char if char in (string.punctuation + string.ascii_lowercase + ' ') else ' ' for char in text))  # keep the selected letters and punctuation.

    # Add start and end tokens
    text = f"{start_token} {text} {end_token}"
    return text

# Apply the cleaning function to each element in the desired columns
data['en_clean'] = data['en'].apply(clean_text)
data['fr_clean'] = data['fr'].apply(clean_text)

# Display the cleaned data
print(data[['en_clean', 'fr_clean']])


                                                 en_clean  \
0                                       <START> hi. <END>   
1                                      <START> run! <END>   
2                                      <START> run! <END>   
3                                      <START> who? <END>   
4                                      <START> wow! <END>   
...                                                   ...   
175616  <START> top-down economics never works, said o...   
175617  <START> a carbon footprint is the amount of ca...   
175618  <START> death is something that we're often di...   
175619  <START> since there are usually multiple websi...   
175620  <START> if someone who doesn't know your backg...   

                                                 fr_clean  
0                                    <START> salut! <END>  
1                                   <START> cours ! <END>  
2                                  <START> courez ! <END>  
3                          

In [119]:
data_en = '\n'.join(data['en_clean'].tolist())
chars_en = sorted(list(set(data_en)))
chars_en

['\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '>',
 '?',
 'A',
 'D',
 'E',
 'N',
 'R',
 'S',
 'T',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [120]:
data_fr = '\n'.join(data['fr_clean'].tolist())
chars_fr = sorted(list(set(data_fr)))
chars_fr

['\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '>',
 '?',
 'A',
 'D',
 'E',
 'N',
 'R',
 'S',
 'T',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [121]:
data

,en,fr,en_clean,fr_clean
0,Hi.,Salut!,<START> hi. <END>,<START> salut! <END>
1,Run!,Cours !,<START> run! <END>,<START> cours ! <END>
2,Run!,Courez !,<START> run! <END>,<START> courez ! <END>
3,Who?,Qui ?,<START> who? <END>,<START> qui ? <END>
4,Wow!,Ça alors !,<START> wow! <END>,<START> ca alors ! <END>
...,...,...,...,...
175616,"Top-down economics never works, said Obama. ""T...","« L'économie en partant du haut vers le bas, ç...","<START> top-down economics never works, said o...",<START> << l'economie en partant du haut vers ...
175617,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...,<START> a carbon footprint is the amount of ca...,<START> une empreinte carbone est la somme de ...
175618,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...,<START> death is something that we're often di...,<START> la mort est une chose qu'on nous decou...
175619,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...,<START> since there are usually multiple websi...,<START> puisqu'il y a de multiples sites web s...


In [122]:
sent_clean_fr = data['fr_clean'].tolist()
sent_clean_en = data['en_clean'].tolist()

In [123]:
from collections import Counter

In [124]:
class Tokenize():
    def __init__(self, word_vocab=None, max_tokens=None, start_token='<START>', end_token='<END>', unk_token='<UNK>', pad_token='<PAD>'):
        self.max_tokens = max_tokens
        self.start_token = start_token
        self.end_token = end_token
        self.unk_token = unk_token
        self.pad_token = pad_token
        self.word_vocab = word_vocab
        if self.word_vocab:
            self.word_vocab = [self.pad_token, self.start_token, self.end_token, self.unk_token] + [word for word,count in word_count.most_common(max_tokens)]
            self.word_to_idx = {w:i for i,w in enumerate(self.word_vocab)}
            self.idx_to_word = {i:w for i,w in enumerate(self.word_vocab)}
            self.len_vocab = len(self.word_vocab)

    def fit(self, texts):

        if type(texts) == str: texts = [texts]

        word_count = Counter(tokenize.word_tokenize(' '.join(texts)))

        if self.max_tokens:
            max_tokens = self.max_tokens -  3
        else:
            max_tokens = self.max_tokens

        self.word_vocab = [self.pad_token, self.start_token, self.end_token, self.unk_token] + [word for word,count in word_count.most_common(max_tokens)]
        self.word_to_idx = {w:i for i,w in enumerate(self.word_vocab)}
        self.idx_to_word = {i:w for i,w in enumerate(self.word_vocab)}
        self.len_vocab = len(self.word_vocab)

    def __call__(self, texts):
        if type(texts) == str: texts = [texts]
        texts_token = [[self.start_token] + tokenize.word_tokenize(text) + [self.end_token] for text in texts]
        texts_token_id = [[self.word_to_idx.get(word, self.word_to_idx[self.unk_token]) for word in sent] for sent in texts_token]

        return texts_token, texts_token_id


In [125]:


sent_clean_fr_train, sent_clean_fr_valid, sent_clean_en_train, sent_clean_en_valid = train_test_split(sent_clean_fr, sent_clean_en, test_size=0.2, random_state=0)

In [126]:
tokenize_fr = Tokenize()
tokenize_fr.fit(sent_clean_fr_train)
train_token_fr, train_id_fr = tokenize_fr(sent_clean_fr_train)
valid_token_fr, valid_id_fr = tokenize_fr(sent_clean_fr_valid)

In [127]:
tokenize_en = Tokenize()
tokenize_en.fit(sent_clean_en_train)
train_token_en, train_id_en = tokenize_en(sent_clean_en_train)
valid_token_en, valid_id_en = tokenize_en(sent_clean_en_valid)
max_len_fr = max((len(text) for text in valid_token_fr))
max_len_fr

60

In [128]:
max_len_en = max((len(text) for text in valid_token_en))
max_len_en

54

In [129]:
max_len = 64

In [130]:
def make_dataset(query, value, max_length=50, return_shift=False, pad_token_id=0):
        if len(query) > max_length:
            query_pad = query[:max_length]
        else:
            query_pad = query + [pad_token_id]*(max_length-len(query))

        if len(value) > max_length:
            value_pad = value[:max_length]
            value_shifted_pad = value[1:max_length+1]
        else:
            value_pad = value[:-1] + [pad_token_id]*(max_length-len(value[:-1]))
            value_shifted_pad = value[1:] + [pad_token_id]*(max_length-len(value[1:]))


        return query_pad, value_pad, value_shifted_pad
def shift(sent, pad_token_id=0):
    pad_token_shifted = sent[1:] + [pad_token_id]
    return pad_token_shifted
query_pad_train, value_pad_train, value_shifted_pad_train = map(np.array,zip(*map(make_dataset, train_id_fr, train_id_en, [max_len]*len(train_id_en))))
query_pad_valid, value_pad_valid, value_shifted_pad_valid = map(np.array,zip(*map(make_dataset, valid_id_fr, valid_id_en, [max_len]*len(train_id_en))))

In [131]:
value_pad_train[0]

array([  1,   4,   6,   5,  17,  15,  40, 320,  13,   4,   7,   5,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [132]:
value_shifted_pad_train[0]

array([  4,   6,   5,  17,  15,  40, 320,  13,   4,   7,   5,   2,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [133]:
fr_vocab = tokenize_fr.len_vocab
en_vocab = tokenize_en.len_vocab

In [134]:
print(fr_vocab, en_vocab)

25768 13292


In [138]:

UNITS = 1024
EPOCHS = 10
BATCH_SIZE = 256

# Define an input sequence and process it.
encoder_inputs = Input(shape=(max_len))
x = Embedding(en_vocab, UNITS)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(UNITS, return_sequences=True, return_state=True, dropout=0.2)(x)
encoder_states = [state_h, state_c]



# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(max_len))
decoder_embeddings = Embedding(fr_vocab, UNITS)(decoder_inputs)



x = LSTM(UNITS, return_sequences=True, dropout=0.2)(decoder_embeddings, initial_state=encoder_states)

decoder_outputs = Dense(fr_vocab)(x)



en_fr_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
en_fr_model.summary()


Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_25 (InputLayer)       [(None, 64)]                 0         []                            
                                                                                                  
 input_26 (InputLayer)       [(None, 64)]                 0         []                            
                                                                                                  
 embedding_24 (Embedding)    (None, 64, 1024)             1361100   ['input_25[0][0]']            
                                                          8                                       
                                                                                                  
 embedding_25 (Embedding)    (None, 64, 1024)             2638643   ['input_26[0][0]']     

In [139]:
# Define an input sequence and process it.
encoder_inputs1 = Input(shape=(max_len))
y = Embedding(fr_vocab, UNITS)(encoder_inputs1)
encoder_outputs1, state_h1, state_c1 = LSTM(UNITS, return_sequences=True, return_state=True, dropout=0.2)(y)
encoder_states1 = [state_h1, state_c1]



# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs1 = Input(shape=(max_len))
decoder_embeddings1 = Embedding(en_vocab, UNITS)(decoder_inputs1)



y = LSTM(UNITS, return_sequences=True, dropout=0.2)(decoder_embeddings1, initial_state=encoder_states1)

decoder_outputs1 = Dense(en_vocab)(y)



fr_en_model = Model([encoder_inputs1, decoder_inputs1], decoder_outputs1)
fr_en_model.summary()

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_27 (InputLayer)       [(None, 64)]                 0         []                            
                                                                                                  
 input_28 (InputLayer)       [(None, 64)]                 0         []                            
                                                                                                  
 embedding_26 (Embedding)    (None, 64, 1024)             2638643   ['input_27[0][0]']            
                                                          2                                       
                                                                                                  
 embedding_27 (Embedding)    (None, 64, 1024)             1361100   ['input_28[0][0]']     

In [ ]:
def masked_loss(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
    loss = loss_fn(y_true, y_pred)

    # Mask off the losses on padding.
    mask = tf.cast(y_true != 0, loss.dtype)
    loss *= mask

    # Return the total.
    return tf.reduce_sum(loss)/tf.reduce_sum(mask)

def masked_acc(y_true, y_pred):
    # Calculate the accuracy for each item in the batch.
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, y_true.dtype)

    match = tf.cast(y_true == y_pred, tf.float32)
    mask = tf.cast(y_true != 0, tf.float32)

    return tf.reduce_sum(match)/tf.reduce_sum(mask)
# Compile & run training
en_fr_model.compile(optimizer='adam', loss=masked_loss, metrics=[masked_acc, masked_loss])
history = en_fr_model.fit([query_pad_train, value_pad_train], value_shifted_pad_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=([query_pad_valid, value_pad_valid], value_shifted_pad_valid)
                   )

Epoch 1/10
549/549 [==============================] - 252s 451ms/step - loss: 2.9248 - masked_acc: 0.5535 - masked_loss: 2.9246 - val_loss: 2.3912 - val_masked_acc: 0.6097 - val_masked_loss: 2.3914
Epoch 2/10
549/549 [==============================] - 248s 452ms/step - loss: 2.2092 - masked_acc: 0.6247 - masked_loss: 2.2091 - val_loss: 2.0809 - val_masked_acc: 0.6334 - val_masked_loss: 2.0811
Epoch 3/10
549/549 [==============================] - 249s 454ms/step - loss: 1.9668 - masked_acc: 0.6424 - masked_loss: 1.9668 - val_loss: 1.9342 - val_masked_acc: 0.6460 - val_masked_loss: 1.9344
Epoch 4/10
318/549 [================>.............] - ETA: 1:34 - loss: 1.8390 - masked_acc: 0.6521 - masked_loss: 1.8390

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['masked_acc'], label='accuracy')
plt.plot(history.history['val_masked_acc'], label='val_accuracy')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('Accuracy')
plt.legend()

en_fr_history = history

In [ ]:
# Save the model in the native Keras format
en_fr_model.save('en_fr_translation_model.keras')

In [ ]:
def masked_loss(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
    loss = loss_fn(y_true, y_pred)

    # Mask off the losses on padding.
    mask = tf.cast(y_true != 0, loss.dtype)
    loss *= mask

    # Return the total.
    return tf.reduce_sum(loss)/tf.reduce_sum(mask)

def masked_acc(y_true, y_pred):
    # Calculate the accuracy for each item in the batch.
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, y_true.dtype)

    match = tf.cast(y_true == y_pred, tf.float32)
    mask = tf.cast(y_true != 0, tf.float32)

    return tf.reduce_sum(match)/tf.reduce_sum(mask)
# Compile & run training
fr_en_model.compile(optimizer='adam', loss=masked_loss, metrics=[masked_acc, masked_loss])
history = fr_en_model.fit([query_pad_train, value_pad_train], value_shifted_pad_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_data=([query_pad_valid, value_pad_valid], value_shifted_pad_valid)
                   )

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['masked_acc'], label='accuracy')
plt.plot(history.history['val_masked_acc'], label='val_accuracy')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('Accuracy')
plt.legend()

fr_en_history = history

In [ ]:
# Save the model in the native Keras format
fr_en_model.save('fr_en_translation_model.keras')

In [ ]:
def pad_seq(seq, max_length=50, pad_token_id=0):
        if len(seq) > max_length:
            pad = seq[:max_length]
        else:
            pad = seq + [pad_token_id]*(max_length-len(seq))

        return pad

In [ ]:
def fr_en_predict(text_fr, model=fr_en_model, return_sent=False, pad_sent=False, print_gen=True):
    if pad_sent:
        sent_id_fr = text_fr
    else:
        clean_text_fr = clean_text(text_fr) #we clean the french sentence
        sent_token_fr, sent_id_fr = tokenize_fr(clean_text_fr) #convert the cleaned sentence into tokens of words and ids
        sent_id_fr = pad_seq(sent_id_fr[0], max_len) #we pad the sentence
    sent_id_en = np.zeros(max_len, dtype=np.int32) #the input english sentence is an array of 0 of max_len length
    sent_id_en[0] = tokenize_en.word_to_idx[start_token] #we replace the first token by the start_token because that is how our model learned to map the data
    predict = -1
    seq_en = []
    i = 0
    while predict!=2 and i<max_len-1: # 2 is the id of end_token in our dictionary
        softmax = tf.keras.layers.Softmax() #the output of the models are logits so we need to use a softmax layer
        test_predict = softmax(model.predict([np.expand_dims(sent_id_fr, axis=0),  np.expand_dims(sent_id_en, axis=0)], verbose=False)) #we produce the sequences output
        predict = np.argmax(test_predict[0][i]) #we only care about the token predicted for the ith token and select the one with the most probability
        seq_en += [predict]
        sent_id_en[i+1] = predict
        i +=1
    if print_gen:
        print(' '.join([tokenize_en.idx_to_word[i] for i in seq_en if i!=2])) #we print the generated sentence without the end_token
    if return_sent:
        return sent_id_en

In [ ]:
def en_fr_predict(text_en, model=en_fr_model, return_sent=False, pad_sent=False, print_gen=True):
    if pad_sent:
        sent_id_en = text_en
    else:
        clean_text_en = clean_text(text_en) #we clean the french sentence
        sent_token_en, sent_id_en = tokenize_en(clean_text_en) #convert the cleaned sentence into tokens of words and ids
        sent_id_en = pad_seq(sent_id_en[0], max_len) #we pad the sentence
    sent_id_fr = np.zeros(max_len, dtype=np.int32) #the input english sentence is an array of 0 of max_len length
    sent_id_fr[0] = tokenize_fr.word_to_idx[start_token] #we replace the first token by the start_token because that is how our model learned to map the data
    predict = -1
    seq_fr = []
    i = 0
    while predict!=2 and i<max_len-1: # 2 is the id of end_token in our dictionary
        softmax = tf.keras.layers.Softmax() #the output of the models are logits so we need to use a softmax layer
        test_predict = softmax(model.predict([np.expand_dims(sent_id_en, axis=0),  np.expand_dims(sent_id_fr, axis=0)], verbose=False)) #we produce the sequences output
        predict = np.argmax(test_predict[0][i]) #we only care about the token predicted for the ith token and select the one with the most probability
        seq_fr += [predict]
        sent_id_fr[i+1] = predict
        i +=1
    if print_gen:
        print(' '.join([tokenize_fr.idx_to_word[i] for i in seq_fr if i!=2])) #we print the generated sentence without the end_token
    if return_sent:
        return sent_id_fr

hi


In [ ]:
text = "hi"
en_fr_predict(text, model= en_fr_model)

< START > me dire il pas < END >


In [ ]:
text = "me dire il pas"
fr_en_predict(text, model=fr_en_model)

greater greater greater slacks demonstrated demonstrated moscow hooky rewritten abandon outskirts reluctant reluctant blondes rendered unsure blondes demonstrated shaping shaping outskirts shaping reluctant brand-name abandon abandon reluctant blondes dishcloth rendered unsure demonstrated shaping shaping transfixed abandon abandon shaping reluctant reluctant blondes dishcloth demonstrated abandon shaping governs demonstrated governs governs rendered demonstrated vices shaping shaping transfixed reluctant reluctant abandon reluctant blondes dishcloth rendered unsure


In [ ]:
from tensorflow.keras.layers import AdditiveAttention, Attention

attention_layer = Attention(use_scale=True)  # You can use either AdditiveAttention or Attention layer.


In [ ]:
# Define an input sequence and process it.
encoder_inputs1 = Input(shape=(max_len))
y = Embedding(fr_vocab, UNITS)(encoder_inputs1)
encoder_outputs1, state_h1, state_c1 = LSTM(UNITS, return_sequences=True, return_state=True, dropout=0.2)(y)
encoder_states1 = [state_h1, state_c1]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs1 = Input(shape=(max_len))
decoder_embeddings1 = Embedding(en_vocab, UNITS)(decoder_inputs1)

# Apply attention to the decoder inputs and encoder outputs
attention_output = attention_layer([decoder_embeddings1, encoder_outputs1])

# Concatenate attention output and decoder inputs
decoder_input_attention = Concatenate(axis=-1)([decoder_embeddings1, attention_output])

y = LSTM(UNITS, return_sequences=True, dropout=0.2)(decoder_input_attention, initial_state=encoder_states1)
decoder_outputs1 = Dense(en_vocab)(y)

attention_model = Model([encoder_inputs1, decoder_inputs1], decoder_outputs1)
attention_model.summary()


In [ ]:
def masked_loss(y_true, y_pred):
    # Calculate the loss for each item in the batch.
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
    loss = loss_fn(y_true, y_pred)

    # Mask off the losses on padding.
    mask = tf.cast(y_true != 0, loss.dtype)
    loss *= mask

    # Return the total.
    return tf.reduce_sum(loss)/tf.reduce_sum(mask)

def masked_acc(y_true, y_pred):
    # Calculate the accuracy for each item in the batch.
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, y_true.dtype)

    match = tf.cast(y_true == y_pred, tf.float32)
    mask = tf.cast(y_true != 0, tf.float32)

    return tf.reduce_sum(match)/tf.reduce_sum(mask)
attention_model.compile(optimizer='adam', loss=masked_loss, metrics=[masked_acc, masked_loss])

# Train the attention model (replace the input data and parameters as needed)
history_attention = attention_model.fit([query_pad_train, value_pad_train], value_shifted_pad_train,
                                        batch_size=BATCH_SIZE,
                                        epochs=EPOCHS,
                                        validation_data=([query_pad_valid, value_pad_valid], value_shifted_pad_valid))

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(history_attention.history['loss'], label='loss')
plt.plot(history_attention.history['val_loss'], label='val_loss')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_attention.history['masked_acc'], label='accuracy')
plt.plot(history_attention.history['val_masked_acc'], label='val_accuracy')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('Accuracy')
plt.legend()


In [ ]:
def fr_en_predict_attention(text_fr, model=attention_model, return_sent=False, pad_sent=False, print_gen=True):
    if pad_sent:
        sent_id_fr = text_fr
    else:
        clean_text_fr = clean_text(text_fr)  # Clean the French sentence
        sent_token_fr, sent_id_fr = tokenize_fr(clean_text_fr)  # Convert the cleaned sentence into tokens of words and ids
        sent_id_fr = pad_seq(sent_id_fr[0], max_len)  # Pad the sentence

    sent_id_en = np.zeros(max_len, dtype=np.int32)  # The input English sentence is an array of 0 of max_len length
    sent_id_en[0] = tokenize_en.word_to_idx[start_token]  # Replace the first token by the start_token because that is how our model learned to map the data

    predict = -1
    seq_en = []
    i = 0
    while predict != 2 and i < max_len - 1:  # 2 is the id of end_token in our dictionary
        softmax = tf.keras.layers.Softmax()  # The output of the models are logits so we need to use a softmax layer
        test_predict = softmax(model.predict([np.expand_dims(sent_id_fr, axis=0), np.expand_dims(sent_id_en, axis=0)], verbose=False))  # We produce the sequences output
        predict = np.argmax(test_predict[0][i])  # We only care about the token predicted for the ith token and select the one with the most probability
        seq_en += [predict]
        sent_id_en[i + 1] = predict
        i += 1

    if print_gen:
        print(' '.join([tokenize_en.idx_to_word[i] for i in seq_en if i != 2]))  # We print the generated sentence without the end_token
    if return_sent:
        return sent_id_en


In [ ]:
text = "cours"
fr_en_predict(text, model=attention_model)

In [ ]:
attention_model.save('fr_en_attention_model_translation_model.keras')

In [ ]:
# Define an input sequence and process it.
encoder_inputs1 = Input(shape=(max_len))
y = Embedding(en_vocab, UNITS)(encoder_inputs1)
encoder_outputs1, state_h1, state_c1 = LSTM(UNITS, return_sequences=True, return_state=True, dropout=0.2)(y)
encoder_states1 = [state_h1, state_c1]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs1 = Input(shape=(max_len))
decoder_embeddings1 = Embedding(fr_vocab, UNITS)(decoder_inputs1)

# Apply attention to the decoder inputs and encoder outputs
attention_output = attention_layer([decoder_embeddings1, encoder_outputs1])

# Concatenate attention output and decoder inputs
decoder_input_attention = Concatenate(axis=-1)([decoder_embeddings1, attention_output])

y = LSTM(UNITS, return_sequences=True, dropout=0.2)(decoder_input_attention, initial_state=encoder_states1)
decoder_outputs1 = Dense(fr_vocab)(y)

attention_model2 = Model([encoder_inputs1, decoder_inputs1], decoder_outputs1)
attention_model2.summary()

In [ ]:
attention_model2.compile(optimizer='adam', loss=masked_loss, metrics=[masked_acc, masked_loss])

# Train the attention model (replace the input data and parameters as needed)
history_attention2 = attention_model2.fit([query_pad_train, value_pad_train], value_shifted_pad_train,
                                        batch_size=BATCH_SIZE,
                                        epochs=EPOCHS,
                                        validation_data=([query_pad_valid, value_pad_valid], value_shifted_pad_valid))

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(history_attention2.history['loss'], label='loss')
plt.plot(history_attention2.history['val_loss'], label='val_loss')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_attention2.history['masked_acc'], label='accuracy')
plt.plot(history_attention2.history['val_masked_acc'], label='val_accuracy')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch #')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
def en_fr_predict_attention(text_en, model=attention_model2, return_sent=False, pad_sent=False, print_gen=True):
    if pad_sent:
        sent_id_en = text_en
    else:
        clean_text_en = clean_text(text_en)  # Clean the French sentence
        sent_token_en, sent_id_en = tokenize_en(clean_text_en)  # Convert the cleaned sentence into tokens of words and ids
        sent_id_en = pad_seq(sent_id_en[0], max_len)  # Pad the sentence

    sent_id_fr = np.zeros(max_len, dtype=np.int32)  # The input English sentence is an array of 0 of max_len length
    sent_id_fr[0] = tokenize_fr.word_to_idx[start_token]  # Replace the first token by the start_token because that is how our model learned to map the data

    predict = -1
    seq_fr = []
    i = 0
    while predict != 2 and i < max_len - 1:  # 2 is the id of end_token in our dictionary
        softmax = tf.keras.layers.Softmax()  # The output of the models are logits so we need to use a softmax layer
        test_predict = softmax(model.predict([np.expand_dims(sent_id_en, axis=0), np.expand_dims(sent_id_fr, axis=0)], verbose=False))  # We produce the sequences output
        predict = np.argmax(test_predict[0][i])  # We only care about the token predicted for the ith token and select the one with the most probability
        seq_fr += [predict]
        sent_id_fr[i + 1] = predict
        i += 1

    if print_gen:
        print(' '.join([tokenize_en.idx_to_word[i] for i in seq_fr if i != 2]))  # We print the generated sentence without the end_token
    if return_sent:
        return sent_id_fr

In [ ]:
attention_model2.save('en_fr_attention_model_translation_model.keras')

In [ ]:
text = "? stay my you do."
en_fr_predict(text, model=attention_model2)

< START > a suis en quelque debarrasser . < END >


#Predictions

In [ ]:
eng_text = "Hi, How are you?"
fre_text = "Salut comment vas-tu?"

In [ ]:
fr_en_predict(fre_text, model=fr_en_model)

< START > is it nearby or you ? < END >


In [ ]:
en_fr_predict(eng_text, model=en_fr_model)

< START > je c'est plus deux vous a ! part . < END >


In [ ]:
fr_en_predict_attention(fre_text, model=attention_model)

< START > hi , how are you ? < END >


In [ ]:
en_fr_predict_attention(eng_text, model=attention_model2)

< START > what it is to be not for me ? < END >


In [ ]:
from langdetect import detect

def detect_language(text):
    try:
        language = detect(text)
        return language
    except:
        return "Language detection failed"

# Input text for language detection
text = input("Enter the text: ")

# Detect the language of the input text
language = detect_language(text)

# Print the detected language
print("Detected Language:", language)

In [ ]:
import langdetect
def identify_language(text):
    try:
        return detect(text)
    except:
        return "Language detection failed"

text = input("Enter the text: ")
model = input("Enter the model (LSTM or LSTM with attention): ")

if identify_language(text) == "en" and model == "LSTM":
    translation = en_fr_predict(text, model=en_fr_model)
elif identify_language(text) == "fr" and model == "LSTM":
    translation = fr_en_predict(text, model=fr_en_model)
elif identify_language(text) == "en" and model == "LSTM with attention":
    translation = en_fr_predict_attention(text, model=attention_model)
elif identify_language(text) == "fr" and model == "LSTM with attention":
    translation = fr_en_predict_attention(text, model=attention_model2)
else:
    translation = "Unsupported language or model type"
print(identify_language(text))
print("Translation:", translation)


Enter the text: Hi
Enter the model (LSTM or LSTM with attention): LSTM
Language detection failed
Translation: Unsupported language or model type


'fi'